In [1]:
import mesa
from wolf_sheep.random_walk import RandomWalker


class Sheep(RandomWalker):
    """
    A sheep that walks around, reproduces (asexually) and gets eaten.

    The init is the same as the RandomWalker.
    """

    energy = None

    def __init__(self, unique_id, pos, model, moore, energy=None):
        super().__init__(unique_id, pos, model, moore=moore)
        self.energy = energy

    def step(self):
        """
        A model step. Move, then eat grass and reproduce.
        """
        self.random_move()
        living = True

        if self.model.grid:
            # Check for wolves
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            wolves = [obj for obj in this_cell if isinstance(obj, Wolf)]
            if len(wolves) > 0:
                wolf_to_run_from = self.random.choice(wolves)
                self.move_away_from(wolf_to_run_from.pos)

        if self.model.grass:
            # Reduce energy
            self.energy -= 1

            # If there is grass available, eat it
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            grass_patch = [obj for obj in this_cell if isinstance(obj, GrassPatch)][0]
            if grass_patch.fully_grown:
                self.energy += self.model.sheep_gain_from_food
                grass_patch.fully_grown = False

            # Death
            if self.energy < 0:
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)
                living = False

        if living and self.random.random() < self.model.sheep_reproduce:
            # Create a new sheep:
            if self.model.grass:
                self.energy /= 2
            lamb = Sheep(
                self.model.next_id(), self.pos, self.model, self.moore, self.energy
            )
            self.model.grid.place_agent(lamb, self.pos)
            self.model.schedule.add(lamb)


class Wolf(RandomWalker):
    """
    A wolf that walks around, reproduces (asexually) and eats sheep.
    """

    energy = None

    def __init__(self, unique_id, pos, model, moore, energy=None):
        super().__init__(unique_id, pos, model, moore=moore)
        self.energy = energy

    def step(self):
        self.random_move()
        self.energy -= 1

        # If there are sheep present, eat one
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        sheep = [obj for obj in this_cell if isinstance(obj, Sheep)]
        if len(sheep) > 0:
            sheep_to_eat = self.random.choice(sheep)
            self.energy += self.model.wolf_gain_from_food

            # Kill the sheep
            self.model.grid.remove_agent(sheep_to_eat)
            self.model.schedule.remove(sheep_to_eat)

        # Death or reproduction
        if self.energy < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            if self.random.random() < self.model.wolf_reproduce:
                # Create a new wolf cub
                self.energy /= 2
                cub = Wolf(
                    self.model.next_id(), self.pos, self.model, self.moore, self.energy
                )
                self.model.grid.place_agent(cub, cub.pos)
                self.model.schedule.add(cub)


class GrassPatch(mesa.Agent):
    """
    A patch of grass that grows at a fixed rate and it is eaten by sheep
    """

    def __init__(self, unique_id, pos, model, fully_grown, countdown):
        """
        Creates a new patch of grass

        Args:
            grown: (boolean) Whether the patch of grass is fully grown or not
            countdown: Time for the patch of grass to be fully grown again
        """
        super().__init__(unique_id, model)
        self.fully_grown = fully_grown
        self.countdown = countdown
        self.pos = pos

    def step(self):
        if not self.fully_grown:
            if self.countdown <= 0:
                # Set as fully grown
                self.fully_grown = True
                self.countdown = self.model.grass_regrowth_time
            else:
                self.countdown -= 1

In [2]:
"""
Wolf-Sheep Predation Model
================================

Replication of the model found in NetLogo:
    Wilensky, U. (1997). NetLogo Wolf Sheep Predation model.
    http://ccl.northwestern.edu/netlogo/models/WolfSheepPredation.
    Center for Connected Learning and Computer-Based Modeling,
    Northwestern University, Evanston, IL.
"""

import mesa

from wolf_sheep.scheduler import RandomActivationByTypeFiltered
from wolf_sheep.agents import Sheep, Wolf, GrassPatch


class WolfSheep(mesa.Model):
    """
    Wolf-Sheep Predation Model
    """

    height = 20
    width = 20

    initial_sheep = 100
    initial_wolves = 50

    sheep_reproduce = 0.04
    wolf_reproduce = 0.05

    wolf_gain_from_food = 20

    grass = False
    grass_regrowth_time = 30
    sheep_gain_from_food = 4

    verbose = False  # Print-monitoring

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(
        self,
        width=20,
        height=20,
        initial_sheep=100,
        initial_wolves=50,
        sheep_reproduce=0.04,
        wolf_reproduce=0.05,
        wolf_gain_from_food=20,
        grass=False,
        grass_regrowth_time=30,
        sheep_gain_from_food=4,
    ):
        """
        Create a new Wolf-Sheep model with the given parameters.

        Args:
            initial_sheep: Number of sheep to start with
            initial_wolves: Number of wolves to start with
            sheep_reproduce: Probability of each sheep reproducing each step
            wolf_reproduce: Probability of each wolf reproducing each step
            wolf_gain_from_food: Energy a wolf gains from eating a sheep
            grass: Whether to have the sheep eat grass for energy
            grass_regrowth_time: How long it takes for a grass patch to regrow
                                 once it is eaten
            sheep_gain_from_food: Energy sheep gain from grass, if enabled.
        """
        super().__init__()
        # Set parameters
        self.width = width
        self.height = height
        self.initial_sheep = initial_sheep
        self.initial_wolves = initial_wolves
        self.sheep_reproduce = sheep_reproduce
        self.wolf_reproduce = wolf_reproduce
        self.wolf_gain_from_food = wolf_gain_from_food
        self.grass = grass
        self.grass_regrowth_time = grass_regrowth_time
        self.sheep_gain_from_food = sheep_gain_from_food

        self.schedule = RandomActivationByTypeFiltered(self)
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)
        self.datacollector = mesa.DataCollector(
            {
                "Wolves": lambda m: m.schedule.get_type_count(Wolf),
                "Sheep": lambda m: m.schedule.get_type_count(Sheep),
                "Grass": lambda m: m.schedule.get_type_count(
                    GrassPatch, lambda x: x.fully_grown
                ),
            }
        )

        # Create sheep:
        for i in range(self.initial_sheep):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.sheep_gain_from_food)
            sheep = Sheep(self.next_id(), (x, y), self, True, energy)
            self.grid.place_agent(sheep, (x, y))
            self.schedule.add(sheep)

        # Create wolves
        for i in range(self.initial_wolves):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.wolf_gain_from_food)
            wolf = Wolf(self.next_id(), (x, y), self, True, energy)
            self.grid.place_agent(wolf, (x, y))
            self.schedule.add(wolf)

        # Create grass patches
        if self.grass:
            for agent, x, y in self.grid.coord_iter():

                fully_grown = self.random.choice([True, False])

                if fully_grown:
                    countdown = self.grass_regrowth_time
                else:
                    countdown = self.random.randrange(self.grass_regrowth_time)

                patch = GrassPatch(self.next_id(), (x, y), self, fully_grown, countdown)
                self.grid.place_agent(patch, (x, y))
                self.schedule.add(patch)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print(
                [
                    self.schedule.time,
                    self.schedule.get_type_count(Wolf),
                    self.schedule.get_type_count(Sheep),
                    self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
                ]
            )

    def run_model(self, step_count=200):

        if self.verbose:
            print("Initial number wolves: ", self.schedule.get_type_count(Wolf))
            print("Initial number sheep: ", self.schedule.get_type_count(Sheep))
            print(
                "Initial number grass: ",
                self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
            )

        for i in range(step_count):
            self.step()

        if self.verbose:
            print("")
            print("Final number wolves: ", self.schedule.get_type_count(Wolf))
            print("Final number sheep: ", self.schedule.get_type_count(Sheep))
            print(
                "Final number grass: ",
                self.schedule.get_type_count(GrassPatch, lambda x: x.fully_grown),
            )

In [3]:
"""
Generalized behavior for random walking, one grid cell at a time.
"""

import mesa


class RandomWalker(mesa.Agent):
    """
    Class implementing random walker methods in a generalized manner.

    Not intended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        moore: If True, may move in all 8 directions.
                Otherwise, only up, down, left, right.
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)
        
    def move_away_from(self, agent_class):
        """
        Move away from a specified type of agent.
        """
        x, y = self.pos
        possible_moves = self.model.grid.get_neighborhood(
            self.pos,
            self.moore,
            True
        )
        move_distances = [
            (self.model.grid.get_distance(pos, agent.pos), pos)
            for pos in possible_moves
            for agent in self.model.grid.get_cell_list_contents([pos])
            if isinstance(agent, agent_class)
        ]
        if move_distances:
            max_distance, new_pos = max(move_distances)
            self.model.grid.move_agent(self, new_pos)
            
    

In [4]:
from typing import Type, Callable

import mesa


class RandomActivationByTypeFiltered(mesa.time.RandomActivationByType):
    """
    A scheduler that overrides the get_type_count method to allow for filtering
    of agents by a function before counting.

    Example:
    >>> scheduler = RandomActivationByTypeFiltered(model)
    >>> scheduler.get_type_count(AgentA, lambda agent: agent.some_attribute > 10)
    """

    def get_type_count(
        self,
        type_class: Type[mesa.Agent],
        filter_func: Callable[[mesa.Agent], bool] = None,
    ) -> int:
        """
        Returns the current number of agents of certain type in the queue that satisfy the filter function.
        """
        count = 0
        for agent in self.agents_by_type[type_class].values():
            if filter_func is None or filter_func(agent):
                count += 1
        return count

In [5]:
import mesa

from wolf_sheep.agents import Wolf, Sheep, GrassPatch
from wolf_sheep.model import WolfSheep


def wolf_sheep_portrayal(agent):
    if agent is None:
        return

    portrayal = {}

    if type(agent) is Sheep:
        portrayal["Shape"] = "wolf_sheep/resources/sheep.png"
        # https://icons8.com/web-app/433/sheep
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 1

    elif type(agent) is Wolf:
        portrayal["Shape"] = "wolf_sheep/resources/wolf.png"
        # https://icons8.com/web-app/36821/German-Shepherd
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 2
        portrayal["text"] = round(agent.energy, 1)
        portrayal["text_color"] = "White"

    elif type(agent) is GrassPatch:
        if agent.fully_grown:
            portrayal["Color"] = ["#00FF00", "#00CC00", "#009900"]
        else:
            portrayal["Color"] = ["#84e184", "#adebad", "#d6f5d6"]
        portrayal["Shape"] = "rect"
        portrayal["Filled"] = "true"
        portrayal["Layer"] = 0
        portrayal["w"] = 1
        portrayal["h"] = 1

    return portrayal


canvas_element = mesa.visualization.CanvasGrid(wolf_sheep_portrayal, 20, 20, 500, 500)
chart_element = mesa.visualization.ChartModule(
    [
        {"Label": "Wolves", "Color": "#AA0000"},
        {"Label": "Sheep", "Color": "#666666"},
        {"Label": "Grass", "Color": "#00AA00"},
    ]
)

model_params = {
    # The following line is an example to showcase StaticText.
    "title": mesa.visualization.StaticText("Parameters:"),
    "grass": mesa.visualization.Checkbox("Grass Enabled", True),
    "grass_regrowth_time": mesa.visualization.Slider("Grass Regrowth Time", 20, 1, 50),
    "initial_sheep": mesa.visualization.Slider(
        "Initial Sheep Population", 100, 10, 300
    ),
    "sheep_reproduce": mesa.visualization.Slider(
        "Sheep Reproduction Rate", 0.04, 0.01, 1.0, 0.01
    ),
    "initial_wolves": mesa.visualization.Slider("Initial Wolf Population", 50, 10, 300),
    "wolf_reproduce": mesa.visualization.Slider(
        "Wolf Reproduction Rate",
        0.05,
        0.01,
        1.0,
        0.01,
        description="The rate at which wolf agents reproduce.",
    ),
    "wolf_gain_from_food": mesa.visualization.Slider(
        "Wolf Gain From Food Rate", 20, 1, 50
    ),
    "sheep_gain_from_food": mesa.visualization.Slider("Sheep Gain From Food", 4, 1, 10),
}

server = mesa.visualization.ModularServer(
    WolfSheep, [canvas_element, chart_element], "Wolf Sheep Predation", model_params
)
server.port = 8521

In [6]:
"""
Testing the RandomWalker by having an ABM composed only of random walker
agents.
"""

from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.TextVisualization import TextVisualization, TextGrid

from wolf_sheep.random_walk import RandomWalker


class WalkerAgent(RandomWalker):
    """
    Agent which only walks around.
    """

    def step(self):
        self.random_move()


class WalkerWorld(Model):
    """
    Random walker world.
    """

    height = 10
    width = 10

    def __init__(self, width, height, agent_count):
        """
        Create a new WalkerWorld.

        Args:
            width, height: World size.
            agent_count: How many agents to create.
        """
        self.height = height
        self.width = width
        self.grid = MultiGrid(self.width, self.height, torus=True)
        self.agent_count = agent_count

        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.agent_count):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a = WalkerAgent(i, (x, y), self, True)
            self.schedule.add(a)
            self.grid.place_agent(a, (x, y))

    def step(self):
        self.schedule.step()


class WalkerWorldViz(TextVisualization):
    """
    ASCII Visualization for a WalkerWorld agent.
    Each cell is displayed as the number of agents currently in that cell.
    """

    def __init__(self, model):
        """
        Create a new visualization for a WalkerWorld instance.

        args:
            model: An instance of a WalkerWorld model.
        """
        self.model = model
        grid_viz = TextGrid(self.model.grid, None)
        grid_viz.converter = lambda x: str(len(x))
        self.elements = [grid_viz]


if __name__ == "__main__":
    print("Testing 10x10 world, with 50 random walkers, for 10 steps.")
    model = WalkerWorld(10, 10, 50)
    viz = WalkerWorldViz(model)
    for i in range(10):
        print("Step:", str(i))
        viz.step()

Testing 10x10 world, with 50 random walkers, for 10 steps.
Step: 0
0020021111
1010100000
1011100100
0000100010
0101011100
0000103001
1010112100
2012013001
0100000002
0000011000

Step: 1
0000001200
0200110001
0102100000
0002012101
0100010000
1000010200
1021122102
1000110010
1000030000
0020001110

Step: 2
0110010000
0110000020
0210211000
0001120001
0000001111
2100001100
0101102111
0100122000
0000002011
0000100210

Step: 3
2000010200
0000000000
1121001011
0001020101
0110200020
0100010110
1110031010
0000111001
0000031101
1100100001

Step: 4
0001000000
2201011100
0010000000
0112011121
0000000120
0021101011
0110111000
2000200200
1000001001
1100111100

Step: 5
1110011000
0110000010
0102110101
0000010111
1101101200
1120001200
0100001000
0003000101
2000000101
1001110110

Step: 6
1201201001
0101100001
0020001001
0011100120
1001000110
2100011001
0013001100
1100100101
0010001101
0000010101

Step: 7
3001011000
0000020000
2201110010
1100012100
0111000001
0102000101
1000231010
0011000010
1000000101
2

In [12]:
!python run.py

In [14]:
import random
import mesa
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

class Sheep(mesa.Agent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, model)
        self.pos = pos

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        self.move()

class Wolf(mesa.Agent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, model)
        self.pos = pos

    def move(self):
        sheep_agents = self.model.schedule.agents_by_type[Sheep]
        if len(sheep_agents) > 0:
            # Find the closest sheep
            closest_sheep = min(sheep_agents, key=lambda a: self.distance(a.pos))
            # Move towards the sheep
            dx = 1 if closest_sheep.pos[0] > self.pos[0] else -1
            dy = 1 if closest_sheep.pos[1] > self.pos[1] else -1
            new_position = (self.pos[0] + dx, self.pos[1] + dy)
        else:
            # If there are no sheep, move randomly
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def distance(self, pos):
        return abs(self.pos[0] - pos[0]) + abs(self.pos[1] - pos[1])

    def step(self):
        self.move()

class WolfPredationModel(mesa.Model):
    def __init__(self, num_sheep, num_wolves, width, height):
        self.num_sheep = num_sheep
        self.num_wolves = num_wolves
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create sheep agents
        for i in range(self.num_sheep):
            pos = (random.randrange(width), random.randrange(height))
            sheep = Sheep(i, pos, self)
            self.grid.place_agent(sheep, pos)
            self.schedule.add(sheep)

        # Create wolf agents
        for i in range(self.num_wolves):
            pos = (random.randrange(width), random.randrange(height))
            wolf = Wolf(i, pos, self)
            self.grid.place_agent(wolf, pos)
            self.schedule.add(wolf)

        # Collect data
        self.datacollector = DataCollector(
            {"Sheep": lambda m: m.schedule.get_breed_count(Sheep),
             "Wolves": lambda m: m.schedule.get_breed_count(Wolf)}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [19]:
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.modules import ChartModule

from model.py import WolfPredationModel
from wolf_predation_agent import Sheep, Wolf

# Define the grid size
grid_size = 20

# Create a canvas grid to visualize the model
canvas_element = CanvasGrid(lambda x: x.get_visual(), grid_size, grid_size, 500, 500)

# Create a chart to visualize the wolf and sheep populations
chart_element = ChartModule([{"Label": "Sheep", "Color": "#AA0000"},
                             {"Label": "Wolves", "Color": "#666666"}])

# Create the server with the model, canvas grid, and chart
server = ModularServer(WolfPredationModel, [canvas_element, chart_element], "Wolf Predation Model", {"grid_size": grid_size})

# Run the server
server.port = 8521  # The default
server.launch()


ModuleNotFoundError: No module named 'model'